In [10]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx"
%goworkfix

	- Replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx" already exists.


## Downloading Dataset

The method `mag.Download()` will download the dataset to the data directory, if not yet downloaded.
It then converts the dataset to tensors, which are then available for use.

The tensor is then saved for faster access. After saved, the next call to `mag.Download()` will take 1/2s.

In [2]:
%env GOMLX_PLATFORM=GPU

Set: GOMLX_PLATFORM="GPU"


In [16]:
import (
    "flag"

    // . "github.com/gomlx/gomlx/graph"
    // "github.com/gomlx/gomlx/types/shapes"
    mag "github.com/gomlx/gomlx/examples/ogbnmag"
    "github.com/janpfeifer/must"
)

var (
    flagDataDir   = flag.String("data", "~/work/ogbnmag", "Directory to cache downloaded and generated dataset files.")
)

%%
start := time.Now()
must.M(mag.Download(*flagDataDir))
fmt.Printf("Elapsed: %s\n", time.Since(start))

Elapsed: 558.160595ms


## FNN model, No Graph

The first model will only use the paper features, and no relations. It serves as a baseline.

In a quick experiment, without much hyperparameter tuning, we got 27.27% on test accuracy, which is inline with the corresponding results in [the leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-mag) (the Multi Layer Perceptron MLP entry in the bottom).

In [4]:
import (
	. "github.com/gomlx/gomlx/graph"
	"github.com/gomlx/gomlx/ml/context"
)
var manager = NewManager()

func config() (ctx *context.Context) {
    ctx = context.NewContext(manager)
    ctx.RngStateReset()
    ctx.SetParams(map[string]any{
        "train_steps": 3_000_000,  // ~ 500 epochs
        "batch_size": 128,
        "optimizer": "adamw", 
        optimizers.LearningRateKey: 0.0001,
        layers.L2RegularizationKey: 0.0,
        "normalization": "layer",
        "dropout": 0.05,
        "hidden_layers": 2,
        "num_nodes": 256,
        "plots": true,
    })
    return
}

In [49]:
import "github.com/gomlx/gomlx/examples/ogbnmag/fnn"

%%
must.M(mag.Download(*flagDataDir))
ctx := config()
ctx.SetParams("checkpoint", path.Join(*flagDataDir), "fnn-baseline")
err := fnn.Train(ctx)
if  err != nil {
    fmt.Printf("%+v\n", err)
}


/ :: #rngState:	(UInt64)[3]
/ogbnmag :: PapersEmbeddings:	(Float32)[736389 128]
/ogbnmag :: PapersLabels:	(Int32)[736389 1]


Training (3000000 steps):  100% [========================================] (1762 steps/s) [loss=2.337] [~loss=2.609] [~acc=32.57%]            


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,6.066362e+00 L7.875679e+01,2.516900e+01 L1.162136e+02,3.776392e+01 L1.440187e+02,5.856857e+01 L1.665396e+02,6.405506e+01 L1.858215e+02,9.899647e+01 L2.028344e+02,1.023004e+02 L2.182331e+02,1.049035e+02 L2.324216e+02,1.225644e+02 L2.456637e+02,1.255479e+02 L2.581394e+02,1.446506e+02 L2.699757e+02,1.377223e+02 L2.812988e+02,1.553032e+02 L2.921949e+02,1.521795e+02 L3.027241e+02,1.478944e+02 L3.129289e+02,1.539015e+02 L3.228601e+02,1.606896e+02 L3.325516e+02,1.595282e+02 L3.420417e+02,1.594080e+02 L3.513544e+02,1.643539e+02 L3.605166e+02,1.610300e+02 L3.695435e+02,1.662561e+02 L3.784526e+02,1.698404e+02 L3.872523e+02,1.704411e+02 L3.959534e+02,1.692597e+02 L4.045669e+02,1.757073e+02 L4.131030e+02,1.652750e+02 L4.215709e+02,1.806332e+02 L4.299776e+02,1.779100e+02 L4.383286e+02,1.799123e+02 L4.466318e+02,1.808735e+02 L4.548922e+02,1.800325e+02 L4.631116e+02,1.873411e+02 L4.712965e+02,1.855790e+02 L4.794495e+02,1.820749e+02 L4.875725e+02,1.853388e+02 L4.956692e+02,1.854389e+02 L5.037416e+02,1.878017e+02 L5.117939e+02,1.916863e+02 L5.198273e+02,1.844377e+02 L5.278442e+02,1.890832e+02 L5.358452e+02,1.959113e+02 L5.438325e+02,1.923671e+02 L5.518069e+02,1.980538e+02 L5.597695e+02,1.952505e+02 L5.677217e+02,1.952905e+02 L5.756645e+02,2.030197e+02 L5.835985e+02,1.942894e+02 L5.915245e+02,1.913659e+02 L5.994434e+02,1.978135e+02 L6.073560e+02,2.000963e+02 L6.152623e+02,2.007570e+02 L6.231630e+02,2.014979e+02 L6.310589e+02,2.005768e+02 L6.389503e+02,2.070445e+02 L6.468375e+02,2.032400e+02 L6.547208e+02,2.033601e+02 L6.626008e+02,1.977735e+02 L6.704778e+02,2.023189e+02 L6.783519e+02,2.070645e+02 L6.862235e+02,1.992953e+02 L6.940929e+02,2.067241e+02 L7.019602e+02,2.083260e+02 L7.098256e+02,2.120504e+02 L7.176892e+02,2.058030e+02 L7.255513e+02,2.019184e+02 L7.334118e+02,2.050421e+02 L7.412711e+02,2.065239e+02 L7.491292e+02,2.093872e+02 L7.569861e+02,2.048218e+02 L7.648420e+02,2.017983e+02 L7.726971e+02,2.084461e+02 L7.805514e+02,2.051823e+02 L7.884049e+02,2.093872e+02 L7.962577e+02,2.061634e+02 L8.041099e+02,2.056028e+02 L8.119615e+02,1.983942e+02 L8.198127e+02,2.038207e+02 L8.276633e+02,2.065639e+02 L8.355135e+02,2.030598e+02 L8.433633e+02,2.019184e+02 L8.512128e+02,2.049019e+02 L8.590620e+02,2.075250e+02 L8.663200e+02,2.082659e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L7.875679e+01,2.741141e+01 L1.162136e+02,3.472460e+01 L1.440187e+02,4.529961e+01 L1.665396e+02,4.884619e+01 L1.858215e+02,7.763300e+01 L2.028344e+02,8.438962e+01 L2.182331e+02,8.981303e+01 L2.324216e+02,1.045559e+02 L2.456637e+02,1.099146e+02 L2.581394e+02,1.257318e+02 L2.699757e+02,1.237773e+02 L2.812988e+02,1.348830e+02 L2.921949e+02,1.381319e+02 L3.027241e+02,1.363974e+02 L3.129289e+02,1.413808e+02 L3.228601e+02,1.492894e+02 L3.325516e+02,1.486163e+02 L3.420417e+02,1.502084e+02 L3.513544e+02,1.526030e+02 L3.605166e+02,1.505579e+02 L3.695435e+02,1.555153e+02 L3.784526e+02,1.598385e+02 L3.872523e+02,1.612494e+02 L3.959534e+02,1.606928e+02 L4.045669e+02,1.636051e+02 L4.131030e+02,1.554118e+02 L4.215709e+02,1.688473e+02 L4.299776e+02,1.686661e+02 L4.383286e+02,1.689897e+02 L4.466318e+02,1.723680e+02 L4.548922e+02,1.691191e+02 L4.631116e+02,1.787881e+02 L4.712965e+02,1.742319e+02 L4.794495e+02,1.746979e+02 L4.875725e+02,1.761605e+02 L4.956692e+02,1.761994e+02 L5.037416e+02,1.789822e+02 L5.117939e+02,1.807038e+02 L5.198273e+02,1.782315e+02 L5.278442e+02,1.828913e+02 L5.358452e+02,1.870462e+02 L5.438325e+02,1.872403e+02 L5.518069e+02,1.887030e+02 L5.597695e+02,1.883406e+02 L5.677217e+02,1.884959e+02 L5.756645e+02,1.956926e+02 L5.835985e+02,1.880299e+02 L5.915245e+02,1.846775e+02 L5.994434e+02,1.905280e+02 L6.073560e+02,1.925084e+02 L6.152623e+02,1.949936e+02 L6.231630e+02,1.945018e+02 L6.310589e+02,1.939840e+02 L6.389503e+02,1.992780e+02 L6.468375e+02,1.981389e+02 L6.547208e+02,1.991485e+02 L6.626008e+02,1.916024e+02 L6.704778e+02,1.991227e+02 L6.783519e+02,2.004300e+02 L6.862235e+02,1.9455

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.457721e+02 L7.875679e+01,2.101336e+02 L1.162136e+02,1.906714e+02 L1.440187e+02,1.710360e+02 L1.665396e+02,1.534968e+02 L1.858215e+02,1.359481e+02 L2.028344e+02,1.211466e+02 L2.182331e+02,1.126454e+02 L2.324216e+02,9.981361e+01 L2.456637e+02,9.208359e+01 L2.581394e+02,8.331361e+01 L2.699757e+02,7.917027e+01 L2.812988e+02,7.294770e+01 L2.921949e+02,7.026105e+01 L3.027241e+02,6.688881e+01 L3.129289e+02,6.286447e+01 L3.228601e+02,6.149588e+01 L3.325516e+02,5.554889e+01 L3.420417e+02,5.486125e+01 L3.513544e+02,5.298173e+01 L3.605166e+02,5.179921e+01 L3.695435e+02,4.895250e+01 L3.784526e+02,4.803649e+01 L3.872523e+02,4.255870e+01 L3.959534e+02,4.228375e+01 L4.045669e+02,4.131993e+01 L4.131030e+02,4.248778e+01 L4.215709e+02,3.607413e+01 L4.299776e+02,3.466058e+01 L4.383286e+02,3.368288e+01 L4.466318e+02,3.248998e+01 L4.548922e+02,3.137803e+01 L4.631116e+02,2.872185e+01 L4.712965e+02,2.779285e+01 L4.794495e+02,2.860400e+01 L4.875725e+02,2.535128e+01 L4.956692e+02,2.454388e+01 L5.037416e+02,2.338615e+01 L5.117939e+02,2.161936e+01 L5.198273e+02,2.138631e+01 L5.278442e+02,2.029942e+01 L5.358452e+02,1.911547e+01 L5.438325e+02,1.766497e+01 L5.518069e+02,1.693674e+01 L5.597695e+02,1.613060e+01 L5.677217e+02,1.636573e+01 L5.756645e+02,1.343404e+01 L5.835985e+02,1.396324e+01 L5.915245e+02,1.532509e+01 L5.994434e+02,1.362571e+01 L6.073560e+02,1.109609e+01 L6.152623e+02,1.225933e+01 L6.231630e+02,1.113750e+01 L6.310589e+02,1.050193e+01 L6.389503e+02,9.280691e+00 L6.468375e+02,8.679696e+00 L6.547208e+02,9.001772e+00 L6.626008e+02,8.906671e+00 L6.704778e+02,7.000378e+00 L6.783519e+02,6.336041e+00 L6.862235e+02,8.362938e+00 L6.940929e+02,5.612719e+00 L7.019602e+02,7.417960e+00 L7.098256e+02,5.200000e+00 L7.176892e+02,6.312108e+00 L7.255513e+02,7.316502e+00 L7.334118e+02,5.211372e+00 L7.412711e+02,6.184083e+00 L7.491292e+02,5.997098e+00 L7.569861e+02,6.868630e+00 L7.648420e+02,7.035455e+00 L7.726971e+02,6.608781e+00 L7.805514e+02,5.974709e+00 L7.884049e+02,6.842949e+00 L7.962577e+02,7.106090e+00 L8.041099e+02,7.143649e+00 L8.119615e+02,9.166748e+00 L8.198127e+02,7.542338e+00 L8.276633e+02,8.209714e+00 L8.355135e+02,8.622357e+00 L8.433633e+02,1.026523e+01 L8.512128e+02,9.290974e+00 L8.590620e+02,8.780571e+00 L8.663200e+02,8.033037e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L7.875679e+01,2.240608e+02 L1.162136e+02,2.055478e+02 L1.440187e+02,1.884027e+02 L1.665396e+02,1.719677e+02 L1.858215e+02,1.544566e+02 L2.028344e+02,1.392436e+02 L2.182331e+02,1.292052e+02 L2.324216e+02,1.163667e+02 L2.456637e+02,1.077861e+02 L2.581394e+02,9.883356e+01 L2.699757e+02,9.284984e+01 L2.812988e+02,8.685961e+01 L2.921949e+02,8.210848e+01 L3.027241e+02,7.786859e+01 L3.129289e+02,7.455407e+01 L3.228601e+02,7.150070e+01 L3.325516e+02,6.676623e+01 L3.420417e+02,6.535522e+01 L3.513544e+02,6.335233e+01 L3.605166e+02,6.167615e+01 L3.695435e+02,5.879642e+01 L3.784526e+02,5.713536e+01 L3.872523e+02,5.238830e+01 L3.959534e+02,5.137008e+01 L4.045669e+02,5.066745e+01 L4.131030e+02,5.141504e+01 L4.215709e+02,4.487768e+01 L4.299776e+02,4.345801e+01 L4.383286e+02,4.262761e+01 L4.466318e+02,4.100315e+01 L4.548922e+02,4.035735e+01 L4.631116e+02,3.692469e+01 L4.712965e+02,3.749494e+01 L4.794495e+02,3.626653e+01 L4.875725e+02,3.353591e+01 L4.956692e+02,3.264708e+01 L5.037416e+02,3.115257e+01 L5.117939e+02,3.038845e+01 L5.198273e+02,2.871063e+01 L5.278442e+02,2.771011e+01 L5.358452e+02,2.617115e+01 L5.438325e+02,2.444381e+01 L5.518069e+02,2.432470e+01 L5.597695e+02,2.338132e+01 L5.677217e+02,2.357638e+01 L5.756645e+02,2.048960e+01 L5.835985e+02,2.075231e+01 L5.915245e+02,2.243287e+01 L5.994434e+02,2.079078e+01 L6.073560e+02,1.792368e+01 L6.152623e+02,1.809537e+01 L6.231630e+02,1.786647e+01 L6.310589e+02,1.719922e+01 L6.389503e+02,1.566421e+01 L6.468375e+02,1.486233e+01 L6.547208e+02,1.467281e+01 L6.626008e+02,1.558367e+01 L6.704778e+02,1.300577e+01 L6.783519e+02,1.282077e+01 L6.862235e+02,1.4114

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L7.875679e+01,2.197041e+02 L1.162136e+02,1.990495e+02 L1.440187e+02,1.828476e+02 L1.665396e+02,1.684581e+02 L1.858215e+02,1.561518e+02 L2.028344e+02,1.440565e+02 L2.182331e+02,1.331657e+02 L2.324216e+02,1.243652e+02 L2.456637e+02,1.164626e+02 L2.581394e+02,1.073991e+02 L2.699757e+02,1.037116e+02 L2.812988e+02,9.948708e+01 L2.921949e+02,9.411301e+01 L3.027241e+02,9.217052e+01 L3.129289e+02,8.936984e+01 L3.228601e+02,8.520568e+01 L3.325516e+02,8.103654e+01 L3.420417e+02,7.910791e+01 L3.513544e+02,7.653386e+01 L3.605166e+02,7.511094e+01 L3.695435e+02,7.269280e+01 L3.784526e+02,7.083504e+01 L3.872523e+02,6.819986e+01 L3.959534e+02,6.776694e+01 L4.045669e+02,6.586417e+01 L4.131030e+02,6.294029e+01 L4.215709e+02,6.090910e+01 L4.299776e+02,5.918060e+01 L4.383286e+02,5.757843e+01 L4.466318e+02,5.702786e+01 L4.548922e+02,5.557531e+01 L4.631116e+02,5.629000e+01 L4.712965e+02,5.337672e+01 L4.794495e+02,5.352727e+01 L4.875725e+02,5.357575e+01 L4.956692e+02,5.053776e+01 L5.037416e+02,5.046715e+01 L5.117939e+02,4.850200e+01 L5.198273e+02,4.636567e+01 L5.278442e+02,4.610346e+01 L5.358452e+02,4.339570e+01 L5.438325e+02,4.277862e+01 L5.518069e+02,4.270860e+01 L5.597695e+02,4.119494e+01 L5.677217e+02,3.958242e+01 L5.756645e+02,4.178306e+01 L5.835985e+02,3.927481e+01 L5.915245e+02,3.912923e+01 L5.994434e+02,3.894974e+01 L6.073560e+02,3.442646e+01 L6.152623e+02,3.530143e+01 L6.231630e+02,3.302472e+01 L6.310589e+02,3.451129e+01 L6.389503e+02,3.259848e+01 L6.468375e+02,3.027153e+01 L6.547208e+02,3.009781e+01 L6.626008e+02,2.929273e+01 L6.704778e+02,2.943357e+01 L6.783519e+02,2.737301e+01 L6.862235e+02,2.643627e+01 L6.940929e+02,2.575288e+01 L7.019602e+02,2.404399e+01 L7.098256e+02,2.220401e+01 L7.176892e+02,2.132602e+01 L7.255513e+02,2.189178e+01 L7.334118e+02,2.081852e+01 L7.412711e+02,1.904145e+01 L7.491292e+02,1.855948e+01 L7.569861e+02,1.705406e+01 L7.648420e+02,1.832092e+01 L7.726971e+02,1.611879e+01 L7.805514e+02,1.453705e+01 L7.884049e+02,1.481432e+01 L7.962577e+02,1.365405e+01 L8.041099e+02,1.354749e+01 L8.119615e+02,9.737068e+00 L8.198127e+02,1.296190e+01 L8.276633e+02,1.041240e+01 L8.355135e+02,1.017207e+01 L8.433633e+02,9.059409e+00 L8.512128e+02,8.552927e+00 L8.590620e+02,5.200000e+00 L8.663200e+02,9.114415e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M7.474035e+01,0 L7.474035e+01,-6 M1.081185e+02,0 L1.081185e+02,-6 M1.318007e+02,0 L1.318007e+02,-6 M1.501700e+02,0 L1.501700e+02,-6 M1.651789e+02,0 L1.651789e+02,-6 M1.778686e+02,0 L1.778686e+02,-6 M1.888610e+02,0 L1.888610e+02,-6 M1.985570e+02,0 L1.985570e+02,-6 M2.072304e+02,0 L2.072304e+02,-6 M2.150764e+02,0 L2.150764e+02,-6 M2.642907e+02,0 L2.642907e+02,-6 M2.976689e+02,0 L2.976689e+02,-6 M3.213511e+02,0 L3.213511e+02,-6 M3.397204e+02,0 L3.397204e+02,-6 M3.547292e+02,0 L3.547292e+02,-6 M3.674190e+02,0 L3.674190e+02,-6 M3.784114e+02,0 L3.784114e+02,-6 M3.881074e+02,0 L3.881074e+02,-6 M3.967807e+02,0 L3.967807e+02,-6 M4.538411e+02,0 L4.538411e+02,-6 M4.872193e+02,0 L4.872193e+02,-6 M5.109014e+02,0 L5.109014e+02,-6 M5.292708e+02,0 L5.292708e+02,-6 M5.442796e+02,0 L5.442796e+02,-6 M5.569694e+02,0 L5.569694e+02,-6 M5.679618e+02,0 L5.679618e+02,-6 M5.776578e+02,0 L5.776578e+02,-6 M5.863311e+02,0 L5.863311e+02,-6 M6.433915e+02,0 L6.433915e+02,-6 M6.767696e+02,0 L6.767696e+02,-6 M7.004518e+02,0 L7.004518e+02,-6 M7.188211e+02,0 L7.188211e+02,-6 M7.338300e+02,0 L7.338300e+02,-6 M7.465198e+02,0 L7.465198e+02,-6 M7.575122e+02,0 L7.575122e+02,-6 M7.672081e+02,0 L7.672081e+02,-6 M7.758815e+02,0 L7.758815e+02,-6 M7.837275e+02,0 L7.837275e+02,-6 M8.329418e+02,0 L8.329418e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 900 1100 2000 3000 4000 5000 6000 7000 9000 20000 30000 40000 50000 60000 70000 90000 200000 300000 400000 500000 600000 700000 900000 1100000 2000000 3000000 Steps 3.000 4.000 5.000 loss loss metrics Train: Moving Average Loss

	[Step 3000000] median train step: 458 microseconds

Results on seeds_valid:
	Mean Loss (#loss): 3.002
	Mean Accuracy (#acc): 26.71%
Results on seeds_test:
	Mean Loss (#loss): 2.951
	Mean Accuracy (#acc): 27.37%

